# AODSOC AODS Gainstage Tuning

## Setup

In [ ]:
import time

import matplotlib.pyplot as plt

from naludaq.backend import AcquisitionManager
from naludaq.board import Board, startup_board
from naludaq.communication import AnalogRegisters, ControlRegisters, DigitalRegisters
from naludaq.controllers import get_board_controller, get_readout_controller
from naludaq.controllers import get_dac_controller, get_gainstage_controller
from naludaq.parsers import get_parser
from naludaq.tools.ftdi import list_ftdi_devices
from naludaq.tools.optimizers.gainstagetuner import GainStageTuner

import matplotlib

cmap = matplotlib.cm.get_cmap('viridis')
matplotlib.rcParams.update({"font.size": 18})
matplotlib.rcParams.update({"font.family": "monospace"})

%load_ext autoreload
%autoreload 2

In [ ]:
BOARD_MODEL = "aodsoc_aods"
BOARD = Board(BOARD_MODEL)
BOARD.load_registers()

### Function Definitions

In [ ]:
def get_parsed_event(board, readout_settings: tuple = (1, 1, 1), timeout: int = 3):
    parser = get_parser(board.params)
    (window, lb, wat) = readout_settings
    get_readout_controller(BOARD).set_read_window(window, lb, wat)
    with AcquisitionManager(BOARD).create_temporary() as acq:
        acq.set_output()
        bc = get_board_controller(BOARD)
        bc.start_readout(
            trig="imm",
            lb="forced",
            singleEv=True,
        )
        time.sleep(1)
        bc.stop_readout()
        evt = acq[0]
    parsed_evt = parser.parse(evt)
    return parsed_evt

## Startup Board + Useful Objects

In [ ]:
print(list_ftdi_devices())

In [ ]:
possible_bauds = BOARD.params["possible_bauds"].keys()
speeds = sorted(possible_bauds, reverse=True)
FASTEST = speeds[-1]
BOARD.start_server(output_dir="./")
BOARD.connect_d2xx(serial_number="210357B45D4FB", baud=FASTEST)
startup_board(BOARD)

AR = AnalogRegisters(BOARD)
CR = ControlRegisters(BOARD)
DR = DigitalRegisters(BOARD)

## Set Gainstage

In [ ]:
GSC = get_gainstage_controller(BOARD, chip_number=0)
GSC.ch0_external_input()
GSC.ch1_8x_ch0()
GSC.ch2_8x_ch1()
GSC.ch3_8x_ch2()
gains = GSC.compute_gains()
print(
    f"CHIP 0:"
    f"ch0=ch{gains[0][0]} x {gains[0][1]}\n",
    f"ch1=ch{gains[1][0]} x {gains[1][1]}\n",
    f"ch2=ch{gains[2][0]} x {gains[2][1]}\n",
    f"ch3=ch{gains[3][0]} x {gains[3][1]}\n",
    f"ext=ch{gains[4][0]} x {gains[4][1]}",
)

In [ ]:
GSC = get_gainstage_controller(BOARD, chip_number=1)
GSC.ch0_external_input()
GSC.ch1_8x_ch0()
GSC.ch2_8x_ch1()
GSC.ch3_8x_ch2()
gains = GSC.compute_gains()
print(
    f"CHIP 1:"
    f"ch0=ch{gains[0][0]} x {gains[0][1]}\n",
    f"ch1=ch{gains[1][0]} x {gains[1][1]}\n",
    f"ch2=ch{gains[2][0]} x {gains[2][1]}\n",
    f"ch3=ch{gains[3][0]} x {gains[3][1]}\n",
    f"ext=ch{gains[4][0]} x {gains[4][1]}",
)

## Plot Before Tuning

In [ ]:
evt = get_parsed_event(BOARD, (2,2,2))

In [ ]:
fig = plt.figure(figsize=(8,5), constrained_layout=True)
fig.clear()
plt.xlabel("Sample Number")
plt.ylabel("ADC Counts")
plt.title(f"Before Tuning")
data = evt["data"]
for channel, data in enumerate(data):
    color = cmap(channel/8)
    plt.plot(data, ".-", ms=8, color=color, label=f"Channel {channel}")
plt.legend(ncols=2, fontsize=10, loc="upper right")
plt.grid()
plt.show()

## Gainstage Tuning

In [ ]:
DAC_BOUNDS = [2000, 2300]
ISEL_BOUNDS = [2520, 2750]
BOUNDS = {'x': DAC_BOUNDS, 'y': ISEL_BOUNDS}
GST = GainStageTuner(BOARD, BOUNDS)
GST.chip = 0
GST.events_per_probe = 2
GST.run(n_iter=50)
tuning_0 = GST.tuning

In [ ]:
DAC_BOUNDS = [2000, 2300]
ISEL_BOUNDS = [2520, 2750]
BOUNDS = {'x': DAC_BOUNDS, 'y': ISEL_BOUNDS}
GST = GainStageTuner(BOARD, BOUNDS)
GST.chip = 1
GST.events_per_probe = 2
GST.run(n_iter=50)
tuning_1 = GST.tuning

In [ ]:
# # Copy the values for channels and isel into a yml
print(f"channels:")
print(f"0..3: {tuning_0['dac']}")
print(f"4..7: {tuning_1['dac']}")
print(f"isel:\nvalue: [{tuning_0['isel']}, {tuning_1['isel']}]")

### Set new tuning values

In [ ]:
AnalogRegisters(BOARD, 0).write("isel", tuning_0['isel'])
AnalogRegisters(BOARD, 1).write("isel", tuning_1['isel'])
get_dac_controller(BOARD).set_dacs(tuning_0['dac'], [0, 1, 2, 3])
get_dac_controller(BOARD).set_dacs(tuning_1['dac'], [4, 5, 6, 7])

## Plot After Tuning

In [ ]:
evt = get_parsed_event(BOARD, (2,2,2))

In [ ]:
fig = plt.figure(figsize=(8,5), constrained_layout=True)
fig.clear()
plt.xlabel("Sample Number")
plt.ylabel("ADC Counts")
plt.title(f"After Tuning")
data = evt["data"]
for channel, data in enumerate(data):
    color = cmap(channel/8)
    plt.plot(data, ".-", ms=8, color=color, label=f"Channel {channel}")

plt.legend(ncols=2, fontsize=10, loc="upper right")
plt.grid()
plt.show()

In [ ]:
BOARD.disconnect()
BOARD.disconnect_server()
plt.close('all')